In [2]:
import numpy as np
import pandas as pd

In [53]:
train=pd.read_csv('train_emoji.csv',header=None)
test=pd.read_csv('test_emoji.csv',header=None)

In [54]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [55]:
import emoji as emoji

In [56]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [57]:
emoji.emojize(":fork_and_knife:")

'🍴'

In [58]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [66]:
data = train.values
for i in range(10):
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


In [67]:
from keras.utils import to_categorical

In [73]:
XT=train[0]
Xt=test[0]

YT=to_categorical(train[1])
Yt=to_categorical(test[1])

print(XT.shape)
print(Xt.shape)
print(YT.shape)
print(Yt.shape)

(132,)
(56,)
(132, 5)
(56, 5)


In [78]:
embeddings={}
with open('./saved/glove.6B.50d.txt',encoding='utf-8')as f:
    for line in f:
        values=line.split()
        word=values[0]
        coeffs=np.asarray(values[1:],dtype='float32')
        
        #print(word)
        #print(coeffs)
        embeddings[word]=coeffs

In [79]:
def getOutputEmbeddings(X):
    embedding_matrix_output=np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix]=X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx]=embeddings[X[ix][jx].lower()]
    return embedding_matrix_output

In [80]:
emb_XT=getOutputEmbeddings(XT)
emb_Xt=getOutputEmbeddings(Xt)

c:\users\hp\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [84]:
from keras.layers import *
from keras.models import Sequential

In [85]:

model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

W0430 18:45:52.667671  5732 deprecation_wrapper.py:119] From c:\users\hp\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0430 18:45:52.672702  5732 deprecation_wrapper.py:119] From c:\users\hp\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0430 18:45:53.125277  5732 deprecation_wrapper.py:119] From c:\users\hp\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0430 18:45:53.144304  5732 deprecation.py:506] From c:\users\hp\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions fo

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________________________


In [96]:

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.fit(emb_XT,YT,batch_size=32,epochs=40,shuffle=True,validation_split=0.1)

Train on 118 samples, validate on 14 samples
Epoch 1/40
118/118 [==============================] - 5s 46ms/step - loss: 0.1750 - acc: 0.9661 - val_loss: 0.7860 - val_acc: 0.8571
Epoch 2/40
118/118 [==============================] - 0s 1ms/step - loss: 0.2688 - acc: 0.9068 - val_loss: 0.9835 - val_acc: 0.7857
Epoch 3/40
118/118 [==============================] - 0s 998us/step - loss: 0.0945 - acc: 0.9746 - val_loss: 2.7259 - val_acc: 0.5000
Epoch 4/40
118/118 [==============================] - 0s 1ms/step - loss: 0.2686 - acc: 0.9153 - val_loss: 1.1667 - val_acc: 0.7143
Epoch 5/40
118/118 [==============================] - 0s 964us/step - loss: 0.0575 - acc: 0.9831 - val_loss: 0.7982 - val_acc: 0.8571
Epoch 6/40
118/118 [==============================] - 0s 973us/step - loss: 0.1162 - acc: 0.9746 - val_loss: 0.8623 - val_acc: 0.8571
Epoch 7/40
118/118 [==============================] - 0s 1ms/step - loss: 0.0752 - acc: 0.9746 - val_loss: 1.0981 - val_acc: 0.7857
Epoch 8/40
118/118 [====

In [97]:
model.evaluate(emb_Xt,Yt)

56/56 [==============================] - 0s 338us/step


[2.5362291165760587, 0.6071428656578064]

In [98]:
pred = model.predict_classes(emb_Xt)

In [99]:
for i in range(30):
    print(' '.join(Xt[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Yt[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😓
😓
he got a raise
😁
⚾
she got me a present
❤️
❤️
ha ha ha it was so funny
😁
😁
he is a good friend
❤️
😁
I am upset
❤️
😓
We had such a lovely dinner tonight
❤️
😁
where is the food
🍴
🍴
Stop making this joke ha ha ha
😁
😁
where is the ball
⚾
⚾
work is hard
😓
😁
This girl is messing with me
😓
❤️
are you serious ha ha
😁
😓
Let us go play baseball
⚾
⚾
This stupid grader is not working
😓
😓
work is horrible
😓
😁
Congratulation for having a baby
😁
😁
stop messing around
😓
😓
any suggestions for dinner
🍴
😁
I love taking breaks
❤️
😁
you brighten my day
😁
😁
I boiled rice
🍴
🍴
she is a bully
😓
❤️
Why are you feeling bad
😓
😓
I am upset
😓
😓
I worked during my birthday
😓
😁
My grandmother is the love of my life
❤️
❤️
enjoy your break
😁
⚾
valentine day is near
❤️
😁
